# Homework 1 - Berkeley STAT 157

Handout 1/22/2017, due 1/29/2017 by 4pm in Git by committing to your repository. Please ensure that you add the TA Git account to your repository.

1. Write all code in the notebook.
1. Write all text in the notebook. You can use MathJax to insert math or generic Markdown to insert figures (it's unlikely you'll need the latter). 
1. **Execute** the notebook and **save** the results.
1. To be safe, print the notebook as PDF and add it to the repository, too. Your repository should contain two files: ``homework1.ipynb`` and ``homework1.pdf``. 

The TA will return the corrected and annotated homework back to you via Git (please give `rythei` access to your repository).

In [1]:
from mxnet import ndarray as nd
import time
import numpy as np

## 1. Speedtest for vectorization

Your goal is to measure the speed of linear algebra operations for different levels of vectorization. You need to use `wait_to_read()` on the output to ensure that the result is computed completely, since NDArray uses asynchronous computation. Please see http://beta.mxnet.io/api/ndarray/_autogen/mxnet.ndarray.NDArray.wait_to_read.html for details. 

1. Construct two matrices $A$ and $B$ with Gaussian random entries of size $4096 \times 4096$. 
1. Compute $C = A B$ using matrix-matrix operations and report the time. 
1. Compute $C = A B$, treating $A$ as a matrix but computing the result for each column of $B$ one at a time. Report the time.
1. Compute $C = A B$, treating $A$ and $B$ as collections of vectors. Report the time.
1. Bonus question - what changes if you execute this on a GPU?

In [ ]:
# 1. Construct two matrices A and B with Gaussian random entries of size  4096×4096. 
A = nd.random.normal(0, 1, shape=(4096, 4096))
B = nd.random.normal(0, 1, shape=(4096, 4096))
print("A is ", A)
print("B is ", B)
# 2. Compute C=AB using matrix-matrix operations and report the time.
tic = time.time()
C = nd.dot(A,B)
C.wait_to_read()
print("C is ", C)
print("Q2 takes", time.time()- tic)

# 3. Compute 𝐶=𝐴𝐵, treating 𝐴 as a matrix but computing the result for each column of 𝐵 one at a time. Report the time.
tic = time.time()
C=nd.empty((4096,4096))
for i in range(4096):
    C[:,i] = nd.dot(A,B[:,i])
C.wait_to_read()
print("C is ", C)
print("Q3 takes", time.time() - tic)

# 4. Compute 𝐶=𝐴𝐵, treating 𝐴 and 𝐵 as collections of vectors. Report the time.
tic = time.time()
C=nd.zeros((4096,4096))
for i in range(4096):
    a = A[:,i].reshape(4096,1)
    b = B[i,]
    C += nd.dot(a,b)
C.wait_to_read()
print("C is ", C)
print("Q4 takes", time.time() - tic)

A is  
[[ 1.1630785   0.4838046   0.29956347 ...  1.6892971   1.0433246
   1.4861063 ]
 [ 1.1784046  -0.90841913 -0.3742974  ...  1.8522055  -1.810564
  -1.0253091 ]
 [ 0.01498137  0.12917036  1.0849217  ... -0.8227321   0.23276122
  -1.4382302 ]
 ...
 [ 0.89425045  0.04806394  0.06638992 ... -0.47666612 -0.812731
   1.0233624 ]
 [ 0.47591528 -0.34719884  0.54473215 ...  0.03410461  0.05183557
  -0.20439205]
 [-0.03636989 -2.5792549  -1.0895498  ...  0.8099311  -1.8143338
   0.9271581 ]]
<NDArray 4096x4096 @cpu(0)>
B is  
[[ 0.9049744  -0.8291842   0.5853669  ... -1.7501848  -0.54217017
   0.9640368 ]
 [-0.3738427   1.5078995  -1.4877979  ... -2.4101987   0.80884403
   1.5183809 ]
 [-0.67060167 -2.0757015   0.1026388  ... -1.1176999  -0.7563753
  -1.0046531 ]
 ...
 [-1.0118686   0.04218809  0.7046732  ...  0.02222571  0.6179356
  -0.9676133 ]
 [ 1.3284602  -0.9398111  -1.621257   ... -0.02329569 -1.4178526
   1.1862245 ]
 [ 1.3548605  -2.0296354   0.7279279  ...  0.43979612  0.06064154

## 2. Semidefinite Matrices

Assume that $A \in \mathbb{R}^{m \times n}$ is an arbitrary matrix and that $D \in \mathbb{R}^{n \times n}$ is a diagonal matrix with nonnegative entries. 

1. Prove that $B = A D A^\top$ is a positive semidefinite matrix. 
1. When would it be useful to work with $B$ and when is it better to use $A$ and $D$?

1. Let x be an arbitrary non-zero vector. $x^{T}Bx = x^{T}ADA^{T}x= (A^{T}x)^{T}DA^{T}x = b^{T}Db = \sum\limits_{i=1}^{n} \lambda_{i} b_{i}^2 >=0$. so B is a PSD 

2. B is eigendecomposed, so we can get a basis of orthonormal eigenvectors, and their eigenvalues are real. When calculating the power of B, it's easy to use $ADA^{T}$

## 3. MXNet on GPUs

1. Install GPU drivers (if needed)
1. Install MXNet on a GPU instance
1. Display `!nvidia-smi`
1. Create a $2 \times 2$ matrix on the GPU and print it. See http://d2l.ai/chapter_deep-learning-computation/use-gpu.html for details.

## 4. NDArray and NumPy 

Your goal is to measure the speed penalty between MXNet Gluon and Python when converting data between both. We are going to do this as follows:

1. Create two Gaussian random matrices $A, B$ of size $4096 \times 4096$ in NDArray. 
1. Compute a vector $\mathbf{c} \in \mathbb{R}^{4096}$ where $c_i = \|A B_{i\cdot}\|^2$ where $\mathbf{c}$ is a **NumPy** vector.

To see the difference in speed due to Python perform the following two experiments and measure the time:

1. Compute $\|A B_{i\cdot}\|^2$ one at a time and assign its outcome to $\mathbf{c}_i$ directly.
1. Use an intermediate storage vector $\mathbf{d}$ in NDArray for assignments and copy to NumPy at the end.

In [ ]:
A = nd.array(nd.random.normal(shape=(4096,4096)))
B = nd.array(nd.random.normal(shape=(4096,4096)))
# 1. Compute ∥ABi∥2 one at a time and assign its outcome to ci directly.
start = time.time()
c = np.zeros((4096,1))
for i in range(4096):
    c[i] = nd.norm(nd.dot(A, B[:,i])).asscalar()
end = time.time()
print("Method 1 takes ", end - start)

# 2. Use an intermediate storage d in NDArray for assignments and copy to NumPy at the end.
start = time.time()
c = np.zeros((4096,1))
d = nd.zeros((4096,1))
for i in range(4096):
    d[i] = nd.norm(nd.dot(A, B[:,i])).asscalar()
c = d.asnumpy()
end = time.time()
print("Method 2 takes ", end - start)

## 5. Memory efficient computation

We want to compute $C \leftarrow A \cdot B + C$, where $A, B$ and $C$ are all matrices. Implement this in the most memory efficient manner. Pay attention to the following two things:

1. Do not allocate new memory for the new value of $C$.
1. Do not allocate new memory for intermediate results if possible.

In [ ]:
#tic = time.time()
C = nd.elemwise_add(nd.dot(A,B),C)
#print(time.time()-tic)
#tic = time.time()
nd.elemwise_add(nd.dot(A,B),C,out=C)
#print(time.time()-tic)

## 6. Broadcast Operations

In order to perform polynomial fitting we want to compute a design matrix $A$ with 

$$A_{ij} = x_i^j$$

Our goal is to implement this **without a single for loop** entirely using vectorization and broadcast. Here $1 \leq j \leq 20$ and $x = \{-10, -9.9, \ldots 10\}$. Implement code that generates such a matrix.

In [ ]:
j=nd.arange(1,21)
x=nd.arange(-10,11).reshape(21,1)
nd.broadcast_power(x,j)